# Publication-Quality Plots

Create journal-ready figures for transit analysis.

**Topics:** Plot styles, report figures, customization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transitkit.core import (
    generate_transit_signal_mandel_agol,
    add_noise,
    find_transits_bls_advanced,
    TransitParameters
)
from transitkit.visualization import setup_publication_style, create_transit_report_figure

## 1. Setup Publication Style

In [ ]:
# Apply AAS journal style (300 DPI)
setup_publication_style(style='aas', dpi=150)  # Use 300 for final figures

# Generate test data
time = np.linspace(0, 30, 2000)
flux = generate_transit_signal_mandel_agol(time, period=5.0, t0=2.5, depth=0.01)
flux = add_noise(flux, 0.001, seed=42)
result = find_transits_bls_advanced(time, flux)

## 2. Create Report Figure

In [ ]:
# Create parameters object
params = TransitParameters(
    period=result['period'],
    period_err=0.0001,
    t0=result['t0'],
    t0_err=0.001,
    depth=result['depth'],
    depth_err=result['depth'] * 0.05,
    duration=result['duration'],
    duration_err=0.005,
    snr=result['snr']
)

# Generate comprehensive report figure
fig = create_transit_report_figure(time, flux, params, bls_result=result)
plt.show()

## 3. Custom Phase-Folded Plot

In [ ]:
# Phase fold
phase = ((time - params.t0) / params.period) % 1
phase[phase > 0.5] -= 1

# Bin data
n_bins = 100
bin_edges = np.linspace(-0.5, 0.5, n_bins + 1)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
binned = [np.nanmedian(flux[(phase >= bin_edges[i]) & (phase < bin_edges[i+1])]) for i in range(n_bins)]
binned_err = [np.nanstd(flux[(phase >= bin_edges[i]) & (phase < bin_edges[i+1])]) / 
              np.sqrt(np.sum((phase >= bin_edges[i]) & (phase < bin_edges[i+1]))) for i in range(n_bins)]

# Publication plot
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(phase, flux, '.', color='gray', ms=1, alpha=0.3, label='Data')
ax.errorbar(bin_centers, binned, yerr=binned_err, fmt='o', color='C0', 
            ms=4, capsize=2, label='Binned')

ax.set_xlabel('Orbital Phase')
ax.set_ylabel('Normalized Flux')
ax.set_title(f'Phase-Folded Transit (P = {params.period:.4f} d)')
ax.legend(loc='lower right')
ax.set_xlim(-0.1, 0.1)

plt.tight_layout()
plt.show()

# Save for publication
# fig.savefig('transit_phase_folded.pdf', bbox_inches='tight', dpi=300)

## 4. Multi-Panel Figure

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# (a) Full light curve
axes[0,0].plot(time, flux, 'k.', ms=0.5)
axes[0,0].set_xlabel('Time (days)')
axes[0,0].set_ylabel('Flux')
axes[0,0].set_title('(a) Light Curve')

# (b) BLS Periodogram
axes[0,1].plot(result['all_periods'], result['all_powers'], 'k-', lw=0.5)
axes[0,1].axvline(result['period'], color='C3', ls='--')
axes[0,1].set_xlabel('Period (days)')
axes[0,1].set_ylabel('BLS Power')
axes[0,1].set_title('(b) Periodogram')
axes[0,1].set_xscale('log')

# (c) Phase-folded
axes[1,0].plot(phase, flux, 'k.', ms=1, alpha=0.3)
axes[1,0].plot(bin_centers, binned, 'C0o', ms=3)
axes[1,0].set_xlabel('Phase')
axes[1,0].set_ylabel('Flux')
axes[1,0].set_title('(c) Phase-Folded')
axes[1,0].set_xlim(-0.15, 0.15)

# (d) Transit zoom
transit_mask = np.abs(phase) < 0.03
axes[1,1].plot(phase[transit_mask] * params.period * 24, flux[transit_mask], 'k.', ms=2)
axes[1,1].set_xlabel('Time from mid-transit (hours)')
axes[1,1].set_ylabel('Flux')
axes[1,1].set_title('(d) Transit Detail')

plt.tight_layout()
plt.show()

# fig.savefig('transit_analysis_figure.pdf', bbox_inches='tight', dpi=300)

## 5. Parameter Table

In [ ]:
# Print LaTeX-ready table
print('\\begin{table}')
print('\\caption{Transit Parameters}')
print('\\begin{tabular}{lcc}')
print('\\hline')
print('Parameter & Value & Unit \\\\')
print('\\hline')
print(f'Period & ${params.period:.6f} \\pm {params.period_err:.6f}$ & days \\\\')
print(f'$T_0$ & ${params.t0:.4f} \\pm {params.t0_err:.4f}$ & BJD \\\\')
print(f'Depth & ${params.depth*1e6:.0f} \\pm {params.depth_err*1e6:.0f}$ & ppm \\\\')
print(f'Duration & ${params.duration*24:.2f} \\pm {params.duration_err*24:.2f}$ & hours \\\\')
print(f'$R_p/R_*$ & ${params.rprs:.4f}$ & -- \\\\')
print(f'SNR & ${params.snr:.1f}$ & -- \\\\')
print('\\hline')
print('\\end{tabular}')
print('\\end{table}')

## Summary

Publication tips:
- Use `setup_publication_style('aas')` for AAS journals
- Save as PDF at 300 DPI: `fig.savefig('fig.pdf', dpi=300)`
- Use consistent color schemes
- Include error bars where possible
- Label panels (a), (b), (c) for multi-panel figures